In [1]:
import os
from sage.manifolds.utilities import ExpressionNice as EN

In [2]:
def formal_diff(f, x):
    if f!=0:
        tempX = SR.temp_var()
        return f.subs({x: tempX}).diff(tempX).subs({tempX: x})
    if f==0:
        return 0

In [3]:
#Definimos variables y funciones
var('eps', latex_name = '\\varepsilon')
var('t')
var('r')
var('th', latex_name = '\\theta')
var('ph', latex_name = '\\varphi')
coords = (t,r,th,ph)

var('aaa,bbb,ccc') #que nos conocemos

h = function('h')(r,th)
k = function('k')(r,th)
v = function('v')(r,th)
omega = function('omega', latex_name='\\omega')(r)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Ne = e^(nu/2)*(1+eps^2*h)
Ve = e^(lamb)*(1+2*eps^2*v)
Ke = r^2*(1+2*eps^2*k)
Nphie = eps*omega

# <center> Tensor de Einstein </center>

In [4]:
gdd = matrix(SR, 4, 4)

gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

guu = 1/gdd

K1 = matrix(SR, 4, 4)
K1[0,3] = -omega*r^2*sin(th)^2
K1[3,0] = -omega*r^2*sin(th)^2

K2 = matrix(SR, 4, 4)
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*v
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

In [5]:
gedd = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        gedd[ii,jj] = gdd[ii,jj] + eps*K1[ii,jj] + 1/2*eps^2*K2[ii,jj]
        
geuu_long = 1/gedd
geuu = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        geuu[ii,jj] = geuu_long[ii,jj].subs(eps==0)\
                    + eps*formal_diff(geuu_long[ii,jj],eps).subs(eps==0)\
                    + 1/2*eps^2*formal_diff(formal_diff(geuu_long[ii,jj],eps),eps).subs(eps==0)

chris = [[[0 for kk in range(4)] for jj in range(4)] for ii in range(4)]
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            chris[ii][jj][kk] = 1/2*sum(geuu[ii,dd]*(formal_diff(gedd[dd,jj],coords[kk])
                                                   + formal_diff(gedd[dd,kk],coords[jj])
                                                   - formal_diff(gedd[jj,kk],coords[dd])) for dd in range(4))

riem = [[[[0 for ll in range(4)] for kk in range(4)] for jj in range(4)] for ii in range(4)]
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            for ll in range(4):
                riem[ii][jj][kk][ll] = (formal_diff(chris[ii][jj][ll],coords[kk])
                                      - formal_diff(chris[ii][jj][kk],coords[ll])
                                      + sum(chris[ii][kk][dd]*chris[dd][ll][jj]
                                          - chris[ii][ll][dd]*chris[dd][kk][jj] for dd in range(4)))

ricci = matrix(SR, 4, 4)
for ii in range(4):
    for kk in range(4):
        ricci[ii,kk] = sum(riem[dd][ii][dd][kk] for dd in range(4))

s_curv = sum(sum(ricci[ii,jj]*geuu[ii,jj] for ii in range(4)) for jj in range(4))

Ge = matrix(SR, 4, 4)
Ge = ricci - 1/2*gedd*s_curv

dGe = matrix(SR, 4, 4)
ddGe = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        dGe[ii,jj] = formal_diff(Ge[ii,jj],eps)
        ddGe[ii,jj] = formal_diff(formal_diff(Ge[ii,jj],eps),eps)

Geud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            Geud[ii,jj] += Ge[jj,kk]*geuu[kk,ii]

dGeud = matrix(SR, 4, 4)
ddGeud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        dGeud[ii,jj] = formal_diff(Geud[ii,jj],eps)
        ddGeud[ii,jj] = formal_diff(formal_diff(Geud[ii,jj],eps),eps)

# <center> Tensor de energía-momento </center>

In [6]:
#Omegas
var('Omegan1', latex_name="\\Omega^{(1)}_{n}")
var('Omegap1', latex_name="\\Omega^{(1)}_{p}")
Omegane = eps*Omegan1
Omegape = eps*Omegap1

Nne = function('Nne', latex_name='N_{n\\varepsilon}')(r,th,eps)
Npe = function('Npe', latex_name='N_{p\\varepsilon}')(r,th,eps)

#Tomando eps=0
Nn = function('Nn', latex_name='N_{n}')(r,th)
Np = function('Np', latex_name='N_{p}')(r,th)

sus_func = [eps==0,
            Nne.subs(eps==0)==Nn,
            Npe.subs(eps==0)==Np]

## <center> Campo $u$ de los neutrones </center>

In [7]:
Nne_paper = 1/sqrt(Ne^2-sin(th)^2*Ke*(Nphie-Omegane)^2)

ueu_paper = [Nne_paper,0,0,Nne_paper*Omegane]

In [8]:
ueu = [Nne,0,0,Nne*Omegane]
ued = [0,0,0,0]

for ii in range(4):
    for jj in range(4):
        ued[ii] += ueu[jj]*gedd[ii,jj]

norm = 0
for ii in range(4):
    norm += ueu[ii]*ued[ii]

Nne_value = sqrt(solve(norm==-1, Nne^2)[0].rhs())

for ii in range(4):
    ueu[ii] = ueu[ii].subs(Nne==Nne_value)
    ued[ii] = ued[ii].subs(Nne==Nne_value)

Nn_value = copy(Nne_value)
for ii in range(len(sus_func)):
    Nn_value = Nn_value.subs(sus_func[ii])
Nn_value = Nn_value.simplify_full()

sus_func.extend([Nn==Nn_value])

uu = copy(ueu)
for ii in range(4):
    for jj in range(len(sus_func)):
        uu[ii] = uu[ii].subs(sus_func[jj])

Nne_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegane)^2)

uu_paper = [Nne_paper,0,0,Omegane*Nne_paper]
for ii in range(4):
    for jj in range(len(sus_func)):
        uu_paper[ii] = uu_paper[ii].subs(sus_func[jj])

uu_check = [x-y for (x,y) in zip(uu, uu_paper)]

show(LatexExpr("u_{\\varepsilon}\\lvert_{\\varepsilon=0} \\rightarrow "), bool(uu_check==[0,0,0,0]))

u_{\varepsilon}\lvert_{\varepsilon=0} \rightarrow  True

In [9]:
dueu, dueu_paper, dueu_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    dueu[ii] = formal_diff(ueu[ii], eps)
    dueu_paper[ii] = formal_diff(ueu_paper[ii], eps)
    dueu_check[ii] = dueu[ii] - dueu_paper[ii]
    for jj in range(len(sus_func)):
        dueu_check[ii] = dueu_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d u_{\\varepsilon}}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(dueu_check==[0,0,0,0]))

\frac{d u_{\varepsilon}}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [10]:
ddueu, ddueu_paper, ddueu_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    ddueu[ii] = formal_diff(dueu[ii], eps)
    ddueu_paper[ii] = formal_diff(dueu_paper[ii], eps)
    ddueu_check[ii] = ddueu[ii] - ddueu_paper[ii]
    for jj in range(len(sus_func)):
        ddueu[ii] = ddueu[ii].subs(sus_func[jj])
        ddueu_paper[ii] = ddueu[ii].subs(sus_func[jj])
        ddueu_check[ii] = ddueu_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d^2 u_{\\varepsilon}}{d \\varepsilon^2}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(ddueu_check==[0,0,0,0]))

\frac{d^2 u_{\varepsilon}}{d \varepsilon^2}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [11]:
uu1 = [0,0,0,0]
uu2 = [0,0,0,0]

for ii in range(4):
    uu1[ii] = formal_diff(ueu[ii],eps)
    uu2[ii] = formal_diff(formal_diff(ueu[ii],eps),eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        uu1[ii] = uu1[ii].subs(sus_func[jj])
        uu2[ii] = uu2[ii].subs(sus_func[jj])

uu1_paper = [0,0,0,0]
uu1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
uu1_paper[3] = e^(-nu/2)*Omegan1

uu2_paper = [0,0,0,0]
uu2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegan1*K1[0,3]+Omegan1^2*r^2*sin(th)^2)
uu2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omegan1)  

uu1_check = [ii-jj for (ii,jj) in zip(uu1, uu1_paper)]
uu2_check = [ii-jj for (ii,jj) in zip(uu2, uu2_paper)]

show(LatexExpr("u^{(1)} \\rightarrow"), uu1_check==[0,0,0,0])
show(LatexExpr("u^{(2)} \\rightarrow"), uu2_check==[0,0,0,0])

u^{(1)} \rightarrow True

u^{(2)} \rightarrow True

## <center> Campo $v$ de los protones </center>

In [12]:
Npe_paper = 1/sqrt(Ne^2-sin(th)^2*Ke*(Nphie-Omegape)^2)

veu_paper = [Npe_paper,0,0,Npe_paper*Omegape]

In [13]:
veu = [Npe,0,0,Npe*Omegape]
ved = [0,0,0,0]

for ii in range(4):
    for jj in range(4):
        ved[ii] += veu[jj]*gedd[ii,jj]

norm = 0
for ii in range(4):
    norm += veu[ii]*ved[ii]

Npe_value = sqrt(solve(norm==-1, Npe^2)[0].rhs())
for ii in range(4):
    veu[ii] = veu[ii].subs(Npe==Npe_value)
    ved[ii] = ved[ii].subs(Npe==Npe_value)

Np_value = copy(Npe_value)
for ii in range(len(sus_func)):
    Np_value = Np_value.subs(sus_func[ii])
Np_value = Np_value.simplify_full()

sus_func.extend([Np==Np_value])

vu = copy(veu)
for ii in range(4):
    for jj in range(len(sus_func)):
        vu[ii] = vu[ii].subs(sus_func[jj])

Npe_paper = 1/sqrt(Ne^2 - sin(th)^2*Ke*(Nphie-Omegape)^2)

vu_paper = [Npe_paper,0,0,Omegape*Npe_paper]
for ii in range(4):
    for jj in range(len(sus_func)):
        vu_paper[ii] = vu_paper[ii].subs(sus_func[jj])

vu_check = [x-y for (x,y) in zip(vu, vu_paper)]

show(LatexExpr("v_{\\varepsilon} \\rightarrow "), bool(vu_check==[0,0,0,0]))

v_{\varepsilon} \rightarrow  True

In [14]:
dveu, dveu_paper, dveu_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    dveu[ii] = formal_diff(veu[ii], eps)
    dveu_paper[ii] = formal_diff(veu_paper[ii], eps)
    dveu_check[ii] = dveu[ii] - dveu_paper[ii]
    for jj in range(len(sus_func)):
        dveu_check[ii] = dveu_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d v_{\\varepsilon}}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(dveu_check==[0,0,0,0]))

\frac{d v_{\varepsilon}}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [15]:
ddveu, ddveu_paper, ddveu_check = [0,0,0,0], [0,0,0,0], [0,0,0,0]

for ii in range(4):
    ddveu[ii] = formal_diff(dveu[ii], eps)
    ddveu_paper[ii] = formal_diff(dveu_paper[ii], eps)
    ddveu_check[ii] = ddveu[ii] - ddveu_paper[ii]
    for jj in range(len(sus_func)):
        ddveu_check[ii] = ddveu_check[ii].subs(sus_func[jj])

show(LatexExpr("\\frac{d^2 v_{\\varepsilon}}{d \\varepsilon^2}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(ddveu_check==[0,0,0,0]))

\frac{d^2 v_{\varepsilon}}{d \varepsilon^2}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [16]:
vu1 = [0,0,0,0]
vu2 = [0,0,0,0]

for ii in range(4):
    vu1[ii] = formal_diff(veu[ii],eps)
    vu2[ii] = formal_diff(formal_diff(veu[ii],eps),eps)

for ii in range(4):
    for jj in range(len(sus_func)):
        vu1[ii] = vu1[ii].subs(sus_func[jj])
        vu2[ii] = vu2[ii].subs(sus_func[jj])

vu1_paper = [0,0,0,0]
vu1_paper[0] = 1/2*e^(-3*nu/2)*K1[0,0]
vu1_paper[3] = e^(-nu/2)*Omegap1

vu2_paper = [0,0,0,0]
vu2_paper[0] = e^(-3*nu/2)*(1/2*K2[0,0]+3/4*e^(-nu)*K1[0,0]+2*Omegap1*K1[0,3]+Omegap1^2*r^2*sin(th)^2)
vu2_paper[3] = e^(-nu/2)*(e^(-nu)*K1[0,0]*Omegap1)  

vu1_check = [ii-jj for (ii,jj) in zip(vu1, vu1_paper)]
vu2_check = [ii-jj for (ii,jj) in zip(vu2, vu2_paper)]

show(LatexExpr("v^{(1)} \\rightarrow"), vu1_check==[0,0,0,0])
show(LatexExpr("v^{(2)} \\rightarrow"), vu2_check==[0,0,0,0])

v^{(1)} \rightarrow True

v^{(2)} \rightarrow True

## <center> Ecuación de Einstein </center>

In [17]:
n = function('n')(r,th,eps)
p = function('p')(r,th,eps)
x = function('x')(r,th,eps)

n0 = function('n0', latex_name="n_0")(r)
p0 = function('p0', latex_name="p_0")(r)

eta = function('eta', latex_name="\\eta")(r,th)
Phi = function('Phi', latex_name="\\Phi")(r,th)

n2 = function('n2')(r,th,eps)
p2 = function('p2')(r,th,eps)
x2 = function('x2')(r,th,eps)

neu = [n*ii for ii in ueu] #eq 13 AC
peu = [p*ii for ii in veu] #eq 13 AC

ned = [n*ii for ii in ued]
ped = [p*ii for ii in ved]

n2_value = (n0*(1 + eps^2*eta))^2
p2_value = (p0*(1 + eps^2*Phi))^2
x2_value = 0
for ii in range(4):
    x2_value -= neu[ii]*ped[ii]

sus_func.insert(0,p==p0+n0*eps^2*Phi)
sus_func.insert(0,n==n0+n0*eps^2*eta)

sus_func.insert(0,formal_diff(x,eps)==formal_diff(sqrt(x2_value),eps))
sus_func.insert(0,formal_diff(p,eps)==formal_diff(p0+p0*eps^2*eta,eps))
sus_func.insert(0,formal_diff(n,eps)==formal_diff(n0+n0*eps^2*eta,eps))

sus_func.insert(0,formal_diff(x,r)==formal_diff(sqrt(x2_value),r))
sus_func.insert(0,formal_diff(p,r)==formal_diff(p0+p0*eps^2*eta,r))
sus_func.insert(0,formal_diff(n,r)==formal_diff(n0+n0*eps^2*eta,r))

sus_func.insert(0,formal_diff(formal_diff(x,eps),eps)==formal_diff(formal_diff(sqrt(x2_value),eps),eps))
sus_func.insert(0,formal_diff(formal_diff(p,eps),eps)==formal_diff(formal_diff(p0+p0*eps^2*eta,eps),eps))
sus_func.insert(0,formal_diff(formal_diff(n,eps),eps)==formal_diff(formal_diff(n0+n0*eps^2*eta,eps),eps))

sus_func.insert(0,formal_diff(x2,eps)==formal_diff(x2_value,eps))
sus_func.insert(0,formal_diff(p2,eps)==formal_diff((p0+p0*eps^2*eta)^2,eps))
sus_func.insert(0,formal_diff(n2,eps)==formal_diff((n0+n0*eps^2*eta)^2,eps))

sus_func.insert(0,formal_diff(formal_diff(x2,eps),eps)==formal_diff(formal_diff(x2_value,eps),eps))
sus_func.insert(0,formal_diff(formal_diff(p2,eps),eps)==formal_diff(formal_diff((p0+p0*eps^2*eta)^2,eps),eps))
sus_func.insert(0,formal_diff(formal_diff(n2,eps),eps)==formal_diff(formal_diff((n0+n0*eps^2*eta)^2,eps),eps))

sus_func.insert(0,x2==x2_value)
sus_func.insert(0,p2==p2_value)
sus_func.insert(0,n2==n2_value)

Lambdae = function('Lambdae', latex_name='\\Lambda_{\\varepsilon}')(n2, p2, x2)

Psie = function('Psie', latex_name='\\Psi_{\\varepsilon}')(n2, p2, x2)

nd = copy(ned)
pd = copy(ped)
for ii in range(4):
    for jj in range(len(sus_func)):
        nd[ii] = nd[ii].subs(sus_func[jj])
        pd[ii] = pd[ii].subs(sus_func[jj])

A = -formal_diff(Lambdae, x2)
B = -2*formal_diff(Lambdae, n2)
C = -2*formal_diff(Lambdae, p2)

mued1 = [B*ii for ii in ned]
mued2 = [A*ii for ii in ped]
chied1 = [C*ii for ii in ped]
chied2 = [A*ii for ii in ned]

mued = [0,0,0,0]
chied = [0,0,0,0]
for ii in range(4):
    mued[ii] = mued1[ii] + mued2[ii]
    chied[ii] = chied1[ii] + chied2[ii]

mueu = [0,0,0,0]
chieu = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        mueu[ii] += mued[jj]*geuu[ii,jj]
        chieu[ii] += chied[jj]*geuu[ii,jj]

A0 = copy(A)
B0 = copy(B)
C0 = copy(C)
for ii in range(len(sus_func)):
    A0 = A0.subs(sus_func[ii])
    B0 = B0.subs(sus_func[ii])
    C0 = C0.subs(sus_func[ii])

## <center> $\mu_c$ $\&$ $\chi_c$ </center>

In [18]:
omegane = sin(th)*sqrt(Ke)*(Nphie-Omegane)/Ne
omegape = sin(th)*sqrt(Ke)*(Nphie-Omegape)/Ne

In [19]:
tu = [1,0,0,0]
td = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        td[ii] += tu[jj]*gedd[ii,jj]

phiu = [0,0,0,1]
phid = [0,0,0,0]
for ii in range(4):
    for jj in range(4):
        phid[ii] += phiu[jj]*gedd[ii,jj]

In [20]:
muecd = [0,0,0,0]
muecd[0] = td[0] + eps*Omegan1*phid[0]
muecd[1] = td[1] + eps*Omegan1*phid[1]
muecd[2] = td[2] + eps*Omegan1*phid[2]
muecd[3] = td[3] + eps*Omegan1*phid[3]

muec = 0
for ii in range(4):
    muec -= muecd[ii]*mueu[ii]

muec_paper = Ne*(B*n*sqrt(1-omegane^2)+A*p*(1-omegane*omegape)/sqrt(1-omegape^2))
muec_check = muec - muec_paper

muc = copy(muec)
muc_paper = copy(muec_paper)
muc_check = copy(muec_check)
for ii in range(len(sus_func)):
    muc = muc.subs(sus_func[ii])
    muc_paper = muc_paper.subs(sus_func[ii])
    muc_check = muc_check.subs(sus_func[ii])
show(LatexExpr("\\mu_c\\lvert_{\\varepsilon=0} \\rightarrow "), bool(muc_check==0))

dmuc = formal_diff(muec,eps)
dmuc_paper = formal_diff(muec_paper,eps)
dmuc_check = formal_diff(muec_check,eps)
for ii in range(len(sus_func)):
    dmuc = dmuc.subs(sus_func[ii])
    dmuc_paper = dmuc_paper.subs(sus_func[ii])
    dmuc_check = dmuc_check.subs(sus_func[ii])
show(LatexExpr("\\frac{d \\mu_c}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(dmuc_check==0))

ddmuc = formal_diff(formal_diff(muec,eps),eps)
ddmuc_paper = formal_diff(formal_diff(muec_paper,eps),eps)
ddmuc_check = formal_diff(formal_diff(muec_check,eps),eps)
for ii in range(len(sus_func)):
    ddmuc = ddmuc.subs(sus_func[ii])
    ddmuc_paper = ddmuc_paper.subs(sus_func[ii])
    ddmuc_check = ddmuc_check.subs(sus_func[ii])
ddmuc_check = ddmuc_check.simplify_full()
show(LatexExpr("\\frac{d^2 \\mu_c}{d \\varepsilon^2}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(ddmuc_check==0))

\mu_c\lvert_{\varepsilon=0} \rightarrow  True

\frac{d \mu_c}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

\frac{d^2 \mu_c}{d \varepsilon^2}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [21]:
chiecd = [0,0,0,0]
chiecd[0] = td[0] + eps*Omegap1*phid[0]
chiecd[1] = td[1] + eps*Omegap1*phid[1]
chiecd[2] = td[2] + eps*Omegap1*phid[2]
chiecd[3] = td[3] + eps*Omegap1*phid[3]

chiec = 0
for ii in range(4):
    chiec -= chiecd[ii]*chieu[ii]

chiec_paper = Ne*(C*p*sqrt(1-omegape^2)+A*n*(1-omegane*omegape)/sqrt(1-omegane^2))
chiec_check = chiec - chiec_paper

chic = copy(chiec)
chic_paper = copy(chiec_paper)
chic_check = copy(chiec_check)
for ii in range(len(sus_func)):
    chic = chic.subs(sus_func[ii])
    chic_paper = chic_paper.subs(sus_func[ii])
    chic_check = chic_check.subs(sus_func[ii])
show(LatexExpr("\\chi_c\\lvert_{\\varepsilon=0} \\rightarrow "), bool(chic_check==0))

dchic = formal_diff(chiec,eps)
dchic_paper = formal_diff(chiec_paper,eps)
dchic_check = formal_diff(chiec_check,eps)
for ii in range(len(sus_func)):
    dchic = dchic.subs(sus_func[ii])
    dchic_paper = dchic_paper.subs(sus_func[ii])
    dchic_check = dchic_check.subs(sus_func[ii])
show(LatexExpr("\\frac{d \\chi_c}{d \\varepsilon}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(dmuc_check==0))

ddchic = formal_diff(formal_diff(chiec,eps),eps)
ddchic_paper = formal_diff(formal_diff(chiec_paper,eps),eps)
ddchic_check = formal_diff(formal_diff(chiec_check,eps),eps)
for ii in range(len(sus_func)):
    ddchic_check = ddchic_check.subs(sus_func[ii])
ddchic_check = ddchic_check.simplify_full()
show(LatexExpr("\\frac{d^2 \\chi_c}{d \\varepsilon^2}\\bigg\\lvert_{\\varepsilon=0} \\rightarrow "),
     bool(ddchic_check==0))

\chi_c\lvert_{\varepsilon=0} \rightarrow  True

\frac{d \chi_c}{d \varepsilon}\bigg\lvert_{\varepsilon=0} \rightarrow  True

\frac{d^2 \chi_c}{d \varepsilon^2}\bigg\lvert_{\varepsilon=0} \rightarrow  True

In [22]:
Psie = Lambdae - sum([ii*jj for (ii,jj) in zip(neu, mued)]) - sum([ii*jj for (ii,jj) in zip(peu, chied)])

LambdaePsie = Psie - Lambdae

dirac = diagonal_matrix([1,1,1,1])

Teud = matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        Teud[ii,jj] = Psie*dirac[ii,jj] + peu[ii]*chied[jj] + neu[ii]*mued[jj]

Te, dTe, ddTe = matrix(SR, 4, 4), matrix(SR, 4, 4), matrix(SR, 4, 4)
for ii in range(4):
    for jj in range(4):
        for kk in range(4):
            Te[ii,jj] += Teud[kk,jj]*gedd[kk,ii]

        dTe[ii,jj] = formal_diff(Te[ii,jj],eps)
        ddTe[ii,jj] = formal_diff(formal_diff(Te[ii,jj],eps),eps)

kappa = 8*pi

In [23]:
j = e^(-(lamb+nu)/2)
M = r*(1-e^(-lamb))/2

dG = copy(dGe)
dT = copy(dTe)
for ii in range(0,len(sus_func)):
    dG = dG.subs(sus_func[ii])
    dT = dT.subs(sus_func[ii])

In [31]:
cosa = formal_diff(B,p2)*n*p
for ii in range(len(sus_func)):
    cosa = cosa.subs(sus_func[ii])
show(cosa)

-2*n0(r)*D[0, 1](Lambdae)(n0(r)^2, p0(r)^2, n0(r)*p0(r))

In [32]:
mue = B*n + A*p #eq 22 C
chie = A*n + C*p #eq 22 C

muinf = -mued[0] #eq 20
chiinf = -chied[0] #eq 20
for ii in range(len(sus_func)):
    muinf = muinf.subs(sus_func[ii])
    chiinf = chiinf.subs(sus_func[ii])

eq_mu = formal_diff(muinf,r)
eq_chi = formal_diff(chiinf,r)
for ii in range(len(sus_func)):
    eq_mu = eq_mu.subs(sus_func[ii])
    eq_chi = eq_chi.subs(sus_func[ii])

A00 = A + 2*formal_diff(B,p2)*n*p + 2*formal_diff(A,n2)*n2  + 2*formal_diff(A,p2)*p2 + formal_diff(A,x2)*n*p
B00 = B + 2*formal_diff(B,n2)*n2  + 4*formal_diff(A,n2)*n*p + formal_diff(A,x2)*p2
C00 = C + 2*formal_diff(C,p2)*p2  + 4*formal_diff(A,p2)*n*p + formal_diff(A,x2)*n2

A000 = copy(A00)
B000 = copy(B00)
C000 = copy(C00)
for ii in range(len(sus_func)):
    A000 = A000.subs(sus_func[ii])
    B000 = B000.subs(sus_func[ii])
    C000 = C000.subs(sus_func[ii])

eq26_1 = A000*formal_diff(p,r) + B000*formal_diff(n,r) + 1/2*(B0*n0 + A0*p0)*formal_diff(nu,r)
eq26_2 = C000*formal_diff(p,r) + A000*formal_diff(n,r) + 1/2*(A0*n0 + C0*p0)*formal_diff(nu,r)
for ii in range(len(sus_func)):
    eq26_1 = eq26_1.subs(sus_func[ii])
    eq26_2 = eq26_2.subs(sus_func[ii])

eq_mu = eq_mu*e^(-nu/2)
eq_mu_check = eq_mu - eq26_1
eq_chi = eq_chi*e^(-nu/2)
eq_chi_check = eq_chi - eq26_2

show(LatexExpr("\\text{Eq 26.1} \\rightarrow"), bool(eq_mu_check==0))
show(LatexExpr("\\text{Eq 26.2} \\rightarrow"), bool(eq_chi_check==0))

\text{Eq 26.1} \rightarrow True

\text{Eq 26.2} \rightarrow True

In [26]:
sys.exit()

SystemExit: 

In [33]:
dp0_value = solve(eq26_1, formal_diff(p0,r))[0].rhs()
for ii in range(len(sus_func)):
    dp0_value = dp0_value.subs(sus_func[ii])
dp0_value = dp0_value.simplify_full()

eq26_2 = eq26_2.subs(formal_diff(p,r).subs(eps==0)==dp0_value)

dn0_value = solve(eq26_2, diff(n0,r))[0].rhs()
for ii in range(len(sus_func)):
    dn0_value = dn0_value.subs(sus_func[ii])
dn0_value = dn0_value.simplify_full()

dp0_value = dp0_value.subs(formal_diff(n,r).subs(eps==0)==dn0_value).simplify_full()

sus_func.extend([formal_diff(n,r).subs(eps==0)==dn0_value,
                 formal_diff(p,r).subs(eps==0)==dp0_value])

# <center> $G_{\mu\nu} = \kappa T_{\mu\nu}$ </center>

In [34]:
Lambda0 = copy(Lambdae)
Psi0 = copy(Psie)
for ii in range(len(sus_func)):
    Lambda0 = Lambda0.subs(sus_func[ii])
    Psi0 = Psi0.subs(sus_func[ii])

G = copy(Ge)
T = copy(Te)
for ii in range(len(sus_func)):
    G = G.subs(sus_func[ii])
    T = T.subs(sus_func[ii])

## <center> $G_{00} = \kappa T_{00}$ </center>

In [35]:
dlamb_value = solve(G[0,0] - kappa*T[0,0] == 0, formal_diff(lamb, r))[0].rhs().simplify_full()
dlamb_paper = (1-e^(lamb))/r - 8*pi*r*e^(lamb)*Lambda0
dlamb_check = dlamb_value - dlamb_paper
show(LatexExpr("\\lambda' \\rightarrow"), bool(dlamb_check==0))

for ii in range(len(sus_func)):
    dlamb_value = dlamb_value.subs(sus_func[ii])
dlamb_value = dlamb_value.simplify_full()

sus_func.extend([formal_diff(lamb,r)==dlamb_value])

\lambda' \rightarrow True

## <center> $G_{11} = \kappa T_{11}$ </center>


In [36]:
dnu_value_11 = solve(G[1,1] - kappa*T[1,1] == 0, formal_diff(nu, r))[0].rhs().simplify_full()
dnu_paper = -(1-e^(lamb))/r + 8*pi*r*e^(lamb)*Psi0
dnu_check_11 = dnu_value_11 - dnu_paper
show(LatexExpr("\\nu' \\rightarrow"), bool(dnu_check_11==0))

for ii in range(len(sus_func)):
    dnu_value_11 = dnu_value_11.subs(sus_func[ii])
dnu_value_11 = dnu_value_11.simplify_full()

sus_func.extend([formal_diff(nu,r)==dnu_value_11])

\nu' \rightarrow True

In [37]:
ddnu_value_11 = formal_diff(dnu_value_11,r).simplify_full()
for ii in range(len(sus_func)):
    ddnu_value_11 = ddnu_value_11.subs(sus_func[ii])

ddnu_value_11 = ddnu_value_11.simplify_factorial()
ddnu_value_11 = ddnu_value_11.simplify_trig()
ddnu_value_11 = ddnu_value_11.simplify_rectform()
ddnu_value_11 = ddnu_value_11.simplify_rational()
ddnu_value_11 = ddnu_value_11.expand_sum()
ddnu_value_11 = ddnu_value_11.simplify_full()

ddnu_paper = 2*r*formal_diff(formal_diff(nu,r),r)\
           + formal_diff(nu,r)*(r*formal_diff(nu,r)-2)\
           - formal_diff(lamb,r)*(2+r*formal_diff(nu,r))\
           + 4/r*(e^(lamb)-1) == 0

ddnu_paper = solve(ddnu_paper, formal_diff(formal_diff(nu,r),r))[0].rhs().simplify_full()
for ii in range(len(sus_func)):
    ddnu_paper = ddnu_paper.subs(sus_func[ii])

ddnu_check_11 = ddnu_value_11 - ddnu_paper
show(LatexExpr("\\nu'' \\rightarrow"), bool(ddnu_check_11==0))

\nu'' \rightarrow False

## <center> $G_{11} = \kappa T_{11}$ y $G_{22} = \kappa T_{22}$ </center>


In [38]:
dnu_value_22 = solve(G[2,2] - kappa*T[2,2] == 0, formal_diff(nu, r))[0].rhs().simplify_full()

dEq_1122 = (dnu_value_11 - dnu_value_22 == 0)*e^(nu)*2*r
dEq_1122 = dEq_1122 + 32*pi*r^2*e^(lamb + nu)*n^2*formal_diff(Lambdae, n^2)\
                    + 32*pi*r^2*e^(lamb + nu)*p^2*formal_diff(Lambdae, p^2)\
                    + 32*pi*r^2*e^(lamb + nu)*x^2*formal_diff(Lambdae, x^2)\
                    - 16*pi*r^2*Lambdae*e^(lamb + nu)\
                    + r*e^(nu)*formal_diff(lamb,r)\
                    - 2*e^(lamb + nu)

for ii in range(len(sus_func)):
    dEq_1122 = dEq_1122.subs(sus_func[ii])

ddnu_value = solve(dEq_1122^2, formal_diff(formal_diff(nu,r),r))[0].rhs().simplify_full()

ddnu_paper = 2*r*formal_diff(formal_diff(nu,r),r)\
           + formal_diff(nu,r)*(r*formal_diff(nu,r)-2)\
           - formal_diff(lamb,r)*(2+r*formal_diff(nu,r))\
           + 4/r*(e^(lamb)-1) == 0

ddnu_paper = solve(ddnu_paper, formal_diff(formal_diff(nu,r),r))[0].rhs()
for ii in range(len(sus_func)):
    ddnu_paper = ddnu_paper.subs(sus_func[ii])
    ddnu_value = ddnu_value.subs(sus_func[ii])

ddnu_paper = ddnu_paper.simplify_full()
ddnu_value = ddnu_value.simplify_full()

ddnu_check = ddnu_value - ddnu_paper
show(LatexExpr("\\nu'' \\rightarrow"), bool(ddnu_check==0))

sus_func.extend([formal_diff(formal_diff(nu,r),r)==ddnu_value])

\nu'' \rightarrow False

In [40]:
show(EN(ddnu_check))

(128*pi^2*r^4*e^(2*lamb(r) + nu(r))*n0(r)^4*(d(Lambdae)/d(n0(r)^2))^2 + 128*pi^2*r^4*e^(2*lamb(r) + nu(r))*p0(r)^4*(d(Lambdae)/d(p0(r)^2))^2 + 128*pi^2*r^4*e^(2*lamb(r) + nu(r))*n0(r)^2*p0(r)^2*(d(Lambdae)/d(n0(r)*p0(r)))^2 - 64*pi^2*r^4*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))^2*e^(2*lamb(r) + nu(r)) - 32*pi*r^2*e^(lamb(r) + nu(r))*n0(r)^2*d(Lambdae)/d(n0(r)^2) + 32*(8*pi^2*r^4*e^(2*lamb(r) + nu(r))*n0(r)^2*d(Lambdae)/d(n0(r)^2) - pi*r^2*e^(lamb(r) + nu(r)))*p0(r)^2*d(Lambdae)/d(p0(r)^2) - 16*(pi*r^2*e^(2*lamb(r)) - pi*r^2*e^lamb(r))*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))*e^nu(r) - (e^(2*lamb(r)) - 1)*e^nu(r) + 32*(8*pi^2*r^4*e^(2*lamb(r) + nu(r))*n0(r)*p0(r)^3*d(Lambdae)/d(p0(r)^2) + (8*pi^2*r^4*e^(2*lamb(r) + nu(r))*n0(r)^3*d(Lambdae)/d(n0(r)^2) - pi*r^2*e^(lamb(r) + nu(r))*n0(r))*p0(r))*d(Lambdae)/d(n0(r)*p0(r)) - 8*sqrt(64*pi^2*r^4*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))^2*e^(2*lamb(r) + 2*nu(r)) - 256*pi*r^2*e^(lamb(r) + 2*nu(r))*n0(r)^2*d(Lambdae)/d(n0(r)^2) - 256*pi*r^2*e^(lamb(r) + 2*nu(r))*p0(r)^2*d(Lambdae)/d(p0(r)^2) - 256*pi*r^2*e^(lamb(r) + 2*nu(r))*n0(r)*p0(r)*d(Lambdae)/d(n0(r)*p0(r)) - 8*r^2*e^(2*nu(r))*d^2(nu)/dr^2 + 16*(pi*r^2*e^(2*lamb(r)) + 5*pi*r^2*e^lamb(r))*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))*e^(2*nu(r)) + (e^(2*lamb(r)) - 6*e^lamb(r) + 9)*e^(2*nu(r)))*(pi*r^2*e^lamb(r)*n0(r)^2*d(Lambdae)/d(n0(r)^2) + pi*r^2*e^lamb(r)*p0(r)^2*d(Lambdae)/d(p0(r)^2) + pi*r^2*e^lamb(r)*n0(r)*p0(r)*d(Lambdae)/d(n0(r)*p0(r))))*e^(-nu(r))/r^2 + (128*pi^2*r^4*e^(2*lamb(r))*n0(r)^4*(d(Lambdae)/d(n0(r)^2))^2 + 128*pi^2*r^4*e^(2*lamb(r))*p0(r)^4*(d(Lambdae)/d(p0(r)^2))^2 + 128*pi^2*r^4*e^(2*lamb(r))*n0(r)^2*p0(r)^2*(d(Lambdae)/d(n0(r)*p0(r)))^2 + 64*pi^2*r^4*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))^2*e^(2*lamb(r)) - 8*(3*pi*r^2*e^(2*lamb(r))*d(Lambdae)/d(n0(r)^2) - 5*pi*r^2*e^lamb(r)*d(Lambdae)/d(n0(r)^2))*n0(r)^2 - 16*(12*pi^2*r^4*e^(2*lamb(r))*n0(r)^2*d(Lambdae)/d(n0(r)^2) - pi*r^2*e^(2*lamb(r)) + pi*r^2*e^lamb(r))*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r)) - 8*(24*pi^2*r^4*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))*e^(2*lamb(r))*p0(r)^2 - (32*pi^2*r^4*e^(2*lamb(r))*n0(r)^2*d(Lambdae)/d(n0(r)^2) - 3*pi*r^2*e^(2*lamb(r)) + 5*pi*r^2*e^lamb(r))*p0(r)^2)*d(Lambdae)/d(p0(r)^2) + 8*(32*pi^2*r^4*e^(2*lamb(r))*n0(r)*p0(r)^3*d(Lambdae)/d(p0(r)^2) - 24*pi^2*r^4*Lambdae(n0(r)^2, p0(r)^2, n0(r)*p0(r))*e^(2*lamb(r))*n0(r)*p0(r) + (32*pi^2*r^4*e^(2*lamb(r))*n0(r)^3*d(Lambdae)/d(n0(r)^2) - (3*pi*r^2*e^(2*lamb(r)) - 5*pi*r^2*e^lamb(r))*n0(r))*p0(r))*d(Lambdae)/d(n0(r)*p0(r)) + e^(2*lamb(r)) - 1)/r^2

# <center> Check $T_{\mu\nu}$ </center>

In [ ]:
Ln = omega - Omegan1
Lp = omega - Omegap1

mu0 = copy(mue)
chi0 = copy(chie)
for ii in range(len(sus_func)):
    mu0 = mu0.subs(sus_func[ii])
    chi0 = chi0.subs(sus_func[ii])

mu0 = mu0.simplify_full()
chi0 = chi0.simplify_full()

A0 = copy(A)
B0 = copy(B)
C0 = copy(C)
for ii in range(len(sus_func)):
    A0 = A0.subs(sus_func[ii])
    B0 = B0.subs(sus_func[ii])
    C0 = C0.subs(sus_func[ii])

A000 = copy(A00)
B000 = copy(B00)
C000 = copy(C00)
for ii in range(len(sus_func)):
    A000 = A000.subs(sus_func[ii])
    B000 = B000.subs(sus_func[ii])
    C000 = C000.subs(sus_func[ii])

In [ ]:
#Te00_paper = Lambda0\
#           - eps^2*mu0*n0*(eta-r^2*sin(th)^2/e^(nu)*Omegan1*Ln)\
#           - eps^2*chi0*p0*(Phi-r^2*sin(th)^2/e^(nu)*Omegap1*Lp)\
#           + eps^2*r^2*sin(th)^2/(2*e^(nu))*A0*n0*p0*(Omegan1-Omegap1)^2
#
#Te00_paper = Te00_paper.simplify_full()
#dTe00_paper = formal_diff(Te00_paper, eps).simplify_full()
#ddTe00_paper = formal_diff(dTe00_paper, eps).simplify_full()
#
#Te00 = Teud[0,0]
#dTe00 = formal_diff(Te00, eps)
#ddTe00 = formal_diff(dTe00, eps)
#
## ZERO ORDER
#Te00_check = Te00 - Te00_paper
#for ii in range(len(sus_func)):
#    Te00_check = Te00_check.subs(sus_func[ii])
#
#show(LatexExpr("T_0^0 \\rightarrow"), bool(Te00_check==0))
#
## FIRST ORDER
#dTe00_check = dTe00 - dTe00_paper
#for ii in range(len(sus_func)):
#    dTe00_check = dTe00_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(1)}}_0^0 \\rightarrow"), bool(dTe00_check==0))
#
## SECOND ORDER
#ddTe00_check = ddTe00 - ddTe00_paper
#for ii in range(len(sus_func)):
#    ddTe00_check = ddTe00_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(2)}}^0_0 \\rightarrow"), bool(ddTe00_check==0))
#show(EN(ddTe00_check.simplify_full()))

In [ ]:
#var('aa,bb,cc')
#Te11_paper = Psi0\
#           + eps^2*n0*(n0*B000+p0*A000)*eta + eps^2*p0*(p0*C000+n0*A000)*Phi\
#           + eps^2*r^2*sin(th)^2/(2*e^(nu))*n0*p0*(A0\
#                                                   +n0*formal_diff(A,n)\
#                                                   +p0*formal_diff(A,p)\
#                                                   +2*n0*p0*formal_diff(A,x^2))*(Omegan1-Omegap1)^2
#Te11 = Teud[1,1]
#
#Te11_paper = Te11_paper.simplify_full()
#dTe11_paper = formal_diff(Te11_paper, eps).simplify_full()
#ddTe11_paper = formal_diff(dTe11_paper, eps).simplify_full()
#
#Te11 = Teud[1,1]
#dTe11 = formal_diff(Te11, eps)
#ddTe11 = formal_diff(dTe11, eps)
#
## ZERO ORDER
#Te11_check = Te11 - Te11_paper
#for ii in range(len(sus_func)):
#    Te11_check = Te11_check.subs(sus_func[ii])
#
#show(LatexExpr("T_1^1 \\rightarrow"), bool(Te11_check==0))
#
## FIRST ORDER
#dTe11_check = dTe11 - dTe11_paper
#for ii in range(len(sus_func)):
#    dTe11_check = dTe11_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(1)}}_1^1 \\rightarrow"), bool(dTe11_check==0))
#
## SECOND ORDER
#ddTe11_check = ddTe11 - ddTe11_paper
#for ii in range(len(sus_func)):
#    ddTe11_check = ddTe11_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(2)}}^1_1 \\rightarrow"), bool(ddTe11_check==0))
#show(EN(ddTe11_check.simplify_full()))

In [ ]:
#Te03_paper = 1/2*eps*r^2*sin(th)^2*(3*Psi0-Lambda0)*(Ln+Omegan1)\
#           - eps*r^2*sin(th)^2*(mu0*n0*Omegan1 + chi0*p0*Omegap1)
#
#Te03_paper = Te03_paper.simplify_full()
#dTe03_paper = formal_diff(Te03_paper, eps).simplify_full()
#ddTe03_paper = formal_diff(dTe03_paper, eps).simplify_full()
#
#Te_scalar = 0
#for ii in range(4):
#    for jj in range(4):
#        Te_scalar += Te[ii,jj]*geuu[ii,jj]
#Te03 = Te[0,3] - 1/2*Te_scalar*gedd[0,3]
#dTe03 = formal_diff(Te03, eps)
#ddTe03 = formal_diff(dTe03, eps)
#
## ZERO ORDER
#Te03_check = Te03 - Te03_paper
#for ii in range(len(sus_func)):
#    Te03_check = Te03_check.subs(sus_func[ii])
#
#show(LatexExpr("T_{03} \\rightarrow"), bool(Te03_check==0))
#
## FIRST ORDER
#dTe03_check = dTe03 - dTe03_paper
#for ii in range(len(sus_func)):
#    dTe03_check = dTe03_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(1)}}_{03} \\rightarrow"), bool(dTe03_check==0))
#
## SECOND ORDER
#ddTe03_check = ddTe03 - ddTe03_paper
#for ii in range(len(sus_func)):
#    ddTe03_check = ddTe03_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(2)}}_{03} \\rightarrow"), bool(ddTe03_check==0))

In [ ]:
#Te2233_paper = aa*eps^2*r^2*sin(th)^2/e^(nu)*(n0*(B0*n0*Omegane + A0*p0*Omegape)*Ln\
#                                          +p0*(C0*p0*Omegape + A0*n0*Omegane)*Lp)
#
#Te2233_paper = Te2233_paper.simplify_full()
#dTe2233_paper = formal_diff(Te2233_paper, eps).simplify_full()
#ddTe2233_paper = formal_diff(dTe2233_paper, eps).simplify_full()
#
#Te2233 = Teud[2,2] - Teud[3,3]
#dTe2233 = formal_diff(Te2233, eps)
#ddTe2233 = formal_diff(dTe2233, eps)
#
## ZERO ORDER
#Te2233_check = Te2233 - Te2233_paper
#for ii in range(len(sus_func)):
#    Te2233_check = Te2233_check.subs(sus_func[ii])
#
#show(LatexExpr("T_2^2-T_3^3 \\rightarrow"), bool(Te2233_check==0))
#
## FIRST ORDER
#dTe2233_check = dTe2233 - dTe2233_paper
#for ii in range(len(sus_func)):
#    dTe2233_check = dTe2233_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(1)}}_2^2-{T^{(1)}}_3^3 \\rightarrow"), bool(dTe2233_check==0))
#
## SECOND ORDER
#ddTe2233_check = ddTe2233 - ddTe2233_paper
#for ii in range(len(sus_func)):
#    ddTe2233_check = ddTe2233_check.subs(sus_func[ii])
#
#show(LatexExpr("{T^{(2)}}_2^2-{T^{(2)}}_3^3 \\rightarrow"), bool(ddTe2233_check==0))
#show(EN(ddTe2233_check.simplify_full()))

# <center> $G^{\prime}_{\mu\nu} = \kappa T^{\prime}_{\mu\nu}$ </center>

In [ ]:
dLambdaePsie = formal_diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    dLambdaePsie = dLambdaePsie.subs(sus_func[ii])

In [ ]:
j = e^(-(lamb+nu)/2)
M = r*(1-e^(-lamb))/2

dG = copy(dGe)
dT = copy(dTe)
for ii in range(0,len(sus_func)):
    dG = dG.subs(sus_func[ii])
    dT = dT.subs(sus_func[ii])

In [ ]:
Lambda1_value = formal_diff(Lambdae, eps)
for ii in range(len(sus_func)):
    Lambda1_value = Lambda1_value.subs(sus_func[ii])

Lambda1_paper = 0
Lambda1_check = Lambda1_value - Lambda1_paper

show(LatexExpr("\\Lambda^{(1)} = "), Lambda1_value, LatexExpr("\\rightarrow"), bool(Lambda1_check==0))

In [ ]:
Psi1_value = formal_diff(Lambdae, eps) + formal_diff(LambdaePsie, eps)
for ii in range(len(sus_func)):
    Psi1_value = Psi1_value.subs(sus_func[ii])

Psi1_paper = 0
Psi1_check = Psi1_value - Psi1_paper

show(LatexExpr("\\Psi^{(1)} = "), Psi1_value, LatexExpr("\\rightarrow"), bool(Psi1_check==0))

In [ ]:
#De dEq el único valor que no es idénticamente nulo es 03 (y 30)
dEq_03 = dG[0,3] - kappa*dT[0,3] == 0

ddomega_value = solve(dEq_03, formal_diff(formal_diff(omega,r),r))[0].rhs().simplify_full()

chi0 = C*p + A*n
for ii in range(len(sus_func)):
    chi0 = chi0.subs(sus_func[ii])

ddomega_paper = 1/(r^4)*formal_diff(r^4*e^(-(lamb+nu)/2)*formal_diff(Ln,r),r)\
              - 16*pi*e^((lamb-nu)/2)*(Psi0-Lambda0)*Ln\
              - 16*pi*e^((lamb-nu)/2)*chi0*p0*(Omegan1-Omegap1) == 0
ddomega_paper = solve(ddomega_paper, formal_diff(formal_diff(omega,r),r))[0].rhs().simplify_full()

for ii in range(len(sus_func)):
    ddomega_value = ddomega_value.subs(sus_func[ii])
    ddomega_paper = ddomega_paper.subs(sus_func[ii])

ddomega_check = ddomega_value - ddomega_paper
for ii in range(len(sus_func)):
    ddomega_check = ddomega_check.subs(sus_func[ii])

ddomega_check = ddomega_check.simplify_full()

show(LatexExpr("\\Delta\\omega'' \\rightarrow "), bool(ddomega_check==0))

# <center> $G^{\prime\prime}_{\mu\nu} = \kappa T^{\prime\prime}_{\mu\nu}$ </center>

In [ ]:
ddLambdaePsie = diff(LambdaePsie, eps, eps)
for ii in range(len(sus_func)):
    ddLambdaePsie = ddLambdaePsie.subs(sus_func[ii])

Lambda2_value = formal_diff(formal_diff(Lambdae, eps),eps)
Psi2_value = Lambda2_value + ddLambdaePsie
for ii in range(len(sus_func)):
    Lambda2_value = Lambda2_value.subs(sus_func[ii])
    Psi2_value = Psi2_value.subs(sus_func[ii])

In [ ]:
ddG = copy(ddGe)
ddT = copy(ddTe)
for ii in range(len(sus_func)):
    ddG = ddG.subs(sus_func[ii])
    ddT = ddT.subs(sus_func[ii])

## <center> Descomposición en $P_l(\cos(\theta))$ </center>

In [ ]:
h0 = function('h0', latex_name='h_0')(r)
v0 = function('v0', latex_name='v_0')(r)
eta0 = function('eta0', latex_name='\\eta_0')(r)
Phi0 = function('Phi0', latex_name='\\Phi_0')(r)

h2 = function('h2', latex_name='h_2')(r)
k2 = function('k2', latex_name='k_2')(r)
v2 = function('v2', latex_name='v_2')(r)
eta2 = function('eta2', latex_name='\\eta_2')(r)
Phi2 = function('Phi2', latex_name='\\Phi_2')(r)

LP1 = function('LP1', latex_name='LP_1')(th)
LP2 = function('LP2', latex_name='LP_2')(th)

lp2 = (3*cos(th)^2-1)/2

In [ ]:
s_v(r,th) = v0+v2*lp2
s_k(r,th) = k2*lp2
s_h(r,th) = h0+h2*lp2
s_eta(r,th) = eta0+eta2*lp2
s_Phi(r,th) = Phi0+Phi2*lp2

sus_descomp = [function('v'), s_v,
               function('k'), s_k,
               function('h'), s_h,
               function('eta', latex_name="\\eta"), s_eta,
               function('Phi', latex_name="\\Phi"), s_Phi]

for ii in range(4):
    for jj in range(4):
        for kk in range(0, len(sus_descomp), 2):
            ddG[ii,jj] = ddG[ii,jj].substitute_function(sus_descomp[kk], sus_descomp[kk+1])
            ddT[ii,jj] = ddT[ii,jj].substitute_function(sus_descomp[kk], sus_descomp[kk+1])

ddG = ddG.simplify_full()
ddT = ddT.simplify_full()

In [ ]:
sus_legendre = [sin(th)^2 == (2/3)*(1-LP2),
                1/sin(th)^2 == 1/((2/3)*(1-LP2)),
                cos(th)^2 == (2*LP2+1)/3,
                1/cos(th)^2 == 1/(2*LP2+1)/3,
                cos(th) == LP1]

### <center> $\boxed{l=0}$ </center>

Ecuación (38)

In [ ]:
Lambda2 = formal_diff(formal_diff(Lambdae, eps),eps).simplify_full()

for ii in range(len(sus_func)):
    Lambda2 = Lambda2.subs(sus_func[ii])
for ii in range(0, len(sus_descomp), 2):
    Lambda2 = Lambda2.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    Lambda2 = Lambda2.subs(sus_legendre[ii])

Lambda20 = Lambda2.subs(LP2==0)

Lambda20_paper = mue*n*eta0 + chie*p*Phi0 + eps^2*r^2/(3*e^(nu))*A*n*p*(Omegan1-Omegap1)^2
Lambda20_paper = -2*Lambda20_paper
for ii in range(len(sus_func)):
    Lambda20_paper = Lambda20_paper.subs(sus_func[ii])

Lambda20_check = (Lambda20 - Lambda20_paper).simplify_full()

show(LatexExpr("\\Lambda^{(2)}_0 \\rightarrow"), bool(Lambda20_check==0))

In [ ]:
sys.exit()

Ecuación (42.1)

In [ ]:
# PROBAR A AISLAR TODO:

In [ ]:
cosa = formal_diff(A,eps)
show(EN(cosa))

In [ ]:
show(EN(x^2))

In [ ]:
var('gamman', latex_name = "\\gamma_n")

gamman = formall_diff(formal_diff(muec_paper,eps),eps)/(2*muinf)
gamman_paper = + B000*n0*eta0/mu0\
               + A000*p0*Phi0/mu0\
               + r^2/(3*e^(nu))*p0/mu0*(A0\
                                        +2*n0*n0*formal_diff(A,n^2).subs(eps==0)\
                                        +n0*p0*formal_diff(A,x^2).subs(eps==0))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^(nu))*Ln^2\
               + h0

for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
gamman = gamman.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gamman = gamman.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gamman = gamman.subs(sus_legendre[ii])

show(EN(gamman.subs(LP2==0).simplify_full()))
show(EN(gamman_paper.simplify_full()))
##gamman_check = (gamman - gamman_paper).simplify_full()
##gamman_check = gamman_check.subs(LP2==0).simplify_full()
##
##show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))
##show(EN(gamman_check.simplify_full()))
#show(EN(gamman.simplify_full()))
#show("========================================================================================================")
#show("========================================================================================================")
#show(EN(gamman_paper.simplify_full()))
#show("========================================================================================================")
#show("========================================================================================================")
#show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))

In [ ]:
gamman = formal_diff(formal_diff(muec,eps),eps)/(2*muinf)
for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
show(EN(gamman.simplify_full()))

In [ ]:
var('gamman', latex_name = "\\gamma_n")

gamman = formal_diff(formal_diff(muec,eps),eps)/(2*muinf)
gamman_paper = + B000*n0*eta0/mu0\
               + A000*p0*Phi0/mu0\
               + r^2/(3*e^(nu))*p0/mu0*(A0\
                                        +2*n0*n0*formal_diff(A,n^2).subs(eps==0)\
                                        +n0*p0*formal_diff(A,x^2).subs(eps==0))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^(nu))*Ln^2\
               + h0

for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
gamman = gamman.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gamman = gamman.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gamman = gamman.subs(sus_legendre[ii])

gamman_check = (gamman - gamman_paper).simplify_full()
gamman_check = gamman_check.subs(LP2==0).simplify_full()

show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))
show(EN(gamman_check.simplify_full()))
#show(EN(gamman.simplify_full()))
#show("========================================================================================================")
#show("========================================================================================================")
show(EN(gamman_paper.simplify_full()))
#show("========================================================================================================")
#show("========================================================================================================")
#show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))

In [ ]:
sys.exit()

In [ ]:
var('gamman', latex_name = "\\gamma_n")

gamman = formal_diff(formal_diff(muec,eps),eps)/(2*muinf)
gamman_paper = + B000*n0*eta0/mu0\
               + A000*p0*Phi0/mu0\
               + r^2/(3*e^(nu))*p0/mu0*(A0\
                                        +2*n0*n0*formal_diff(A,n^2).subs(eps==0)\
                                        +n0*p0*formal_diff(A,x^2).subs(eps==0))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^(nu))*Ln^2\
               + h0

for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
    gamman_paper = gamman_paper.subs(sus_func[ii])

gamman = gamman.simplify_full()
gamman_paper = gamman_paper.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gamman = gamman.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gamman = gamman.subs(sus_legendre[ii])

show(EN(gamman.subs(LP2==0).simplify_full()))

gamman_check = (gamman - gamman_paper).simplify_full()
gamman_check = gamman_check.subs(LP2==0).simplify_full()

show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))
show(EN(gamman_check.simplify_full()))

In [ ]:
var('gamman', latex_name = "\\gamma_n")

gamman = formal_diff(formal_diff(muec,eps),eps)/(2*muinf)
gamman_paper = + B000*n0*eta0/mu0\
               + A000*p0*Phi0/mu0\
               + r^2/(3*e^(nu))*p0/mu0*(A0\
                                        +2*n0*n0*formal_diff(A,n^2).subs(eps==0)\
                                        +n0*p0*formal_diff(A,x^2).subs(eps==0))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^(nu))*Ln^2\
               + h0

for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
    gamman_paper = gamman_paper.subs(sus_func[ii])

gamman = gamman.simplify_full()
gamman_paper = gamman_paper.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gamman = gamman.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gamman = gamman.subs(sus_legendre[ii])

show(EN(gamman.subs(LP2==0).simplify_full()))

gamman_check = (gamman - gamman_paper).simplify_full()
gamman_check = gamman_check.subs(LP2==0).simplify_full()

show(LatexExpr("\\gamma_n \\rightarrow"), bool(gamman_check==0))
show(EN(gamman_check.simplify_full()))

In [ ]:
gamman = formal_diff(formal_diff(muec,eps),eps)/(2*muinf)
for ii in range(len(sus_func)):
    gamman = gamman.subs(sus_func[ii])
gamman = gamman.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gamman = gamman.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gamman = gamman.subs(sus_legendre[ii])

gamman = gamman.simplify_full()
gamman = gamman.subs(LP2==0).simplify_full()
show(EN(gamman))
#gamman_paper = + B000*n0*eta0/mu0\
#               + A000*p0*Phi0/mu0\
#               + r^2/(3*e^(nu))*p0/mu0*(A0\
#                                        +2*n0*n0*formal_diff(A,n^2)\
#                                        +n0*p0*formal_diff(A,x^2))*(Omegan1-Omegap1)^2\
#               - r^2/(3*e^(nu))*Ln^2\
#               + h0
#gamman_check = gamman - gamman_paper
#show(gamman_check.simplify_full())

In [ ]:
sys.exit()

Ecuación (42.2)

In [ ]:
var('gammap', latex_name = "\\gamma_p")

gammap = formal_diff(formal_diff(mue,eps),eps)/muinf

gammap_paper = + C00*p*Phi0/chie\
               + A00*n*eta0/chie\
               + r^2/(3*e^(nu))*n/chie*(A+p*formal_diff(A,p) + n*p*formal_diff(A,x^2))*(Omegan1-Omegap1)^2\
               - r^2/(3*e^nu)*Lp^2 + h0
gammap_check = gammap - gammap_paper
for ii in range(len(sus_func)):
    gammap = gammap.subs(sus_func[ii])
    gammap_paper = gammap_paper.subs(sus_func[ii])
    gammap_check = gammap_check.subs(sus_func[ii])

gammap = gammap.simplify_full()

for ii in range(0,len(sus_descomp),2):
    gammap = gammap.substitute_function(sus_descomp[ii],sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    gammap = gammap.subs(sus_legendre[ii])

gammap = gammap.simplify_full()
gammap = gammap.subs(LP2==0).simplify_full()

gammap_paper = gammap_paper.simplify_full()
gammap_check = gammap_check.simplify_full()

show(LatexExpr("\\gamma_p \\rightarrow"), bool(gammap_check==0))

show(EN(gammap_check))

Ecuación (47)

In [ ]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0]

ddEq_00_paper = 16*pi*r^2/(3*e^(nu))*((Psi0-Lambda0)*Ln^2\
                                      + chie*p*(Omegan1-Omegap1)*(Ln+Lp)\
                                      - A*n*p*(Omegan1-Omegap1)^2)\
              + 8*pi*(-Lambda2)\
              - 2/r^2*formal_diff(r/e^(lamb)*v0,r)\
              + r^2/(6*e^(lamb+nu))*(formal_diff(Ln,r))^2

for ii in range(len(sus_func)):
    ddEq_00 = ddEq_00.subs(sus_func[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_func[ii])
    
for ii in range(0,len(sus_descomp),2):
    ddEq_00 = ddEq_00.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_00_paper = ddEq_00_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_00 = ddEq_00.simplify_full()
ddEq_00_paper = ddEq_00_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_00 = ddEq_00.subs(sus_legendre[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_legendre[ii])

ddEq_00 = ddEq_00.subs(LP2==0)
ddEq_00_paper = ddEq_00_paper.subs(LP2==0)
ddEq_00_check = ddEq_00 - ddEq_00_paper

show(LatexExpr("\\text{Ecuación (47) }\\rightarrow"), bool(ddEq_00_check==0))

show(EN(ddEq_00_check))

Ecuación (49)

In [ ]:
ddEq_11 = ddG[1,1] - kappa*ddT[1,1]

ddEq_11_paper = 2/(r*e^(lamb))*formal_diff(h0,r)\
              - 2/(r*e^(lamb))*(formal_diff(nu,r)+1/r)*v0\
              + r^2/(6*e^(nu+lamb))*formal_diff(Ln,r)^2\
              - 8*pi*(mue*n*gamman + chie*p*gammap\
                      - (Psi0 - Lambda0)*h0 + r^2/(3*e^(nu))*(mue*n*Ln^2 + chie*p*Lp^2)\
                      - r^2/(3*e^(nu))*n*p*A*(Omegan1 - Omegap1)^2)

for ii in range(len(sus_func)):
    ddEq_11 = ddEq_11.subs(sus_func[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_func[ii])
    
for ii in range(0,len(sus_descomp),2):
    ddEq_11 = ddEq_11.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_11_paper = ddEq_11_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_11 = ddEq_11.simplify_full()
ddEq_11_paper = ddEq_11_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_11 = ddEq_11.subs(sus_legendre[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_legendre[ii])

ddEq_11 = ddEq_11.subs(LP2==0)
ddEq_11_paper = ddEq_11_paper.subs(LP2==0)
ddEq_11_check = ddEq_11 - ddEq_11_paper

show(LatexExpr("\\text{Ecuación (49) }\\rightarrow"), bool(ddEq_11_check==0))

show(EN(ddEq_11_check))

### <center> $\boxed{l=2}$ </center>

Ecuación (39)

In [ ]:
Lambda2 = formal_diff(formal_diff(Lambdae, eps),eps).simplify_full()

for ii in range(len(sus_func)):
    Lambda2 = Lambda2.subs(sus_func[ii])
for ii in range(0, len(sus_descomp), 2):
    Lambda2 = Lambda2.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
for ii in range(len(sus_legendre)):
    Lambda2 = Lambda2.subs(sus_legendre[ii])
Lambda22 = (Lambda2 - Lambda2.subs(LP2==0))/LP2

Lambda22_paper = mue*n*eta2 + chie*p*Phi2 - r^2/(3*e^(nu))*A*n*p*(Omegane-Omegape)^2 # OMEGANE-OMEGAPE
Lambda22_paper = -Lambda22_paper
for ii in range(len(sus_func)):
    Lambda22_paper = Lambda22_paper.subs(sus_func[ii])

Lambda22_check = (Lambda22 - Lambda22_paper).simplify_full()
show(LatexExpr("\\Lambda^{(2)}_2 \\rightarrow"), bool(Lambda22_check==0))

Ecuación (43.1)

In [ ]:
Eq_43_1 = formal_diff(formal_diff(mue,eps),eps)/muinf
Eq_43_1_paper = + B00*n*eta2/mu\
                + A00*p*Phi2/mu\
                - r^2/(3*e^(nu))*p/mu*(A+n*formal_diff(A,n) + n*p*formal_diff(A,x^2))*(Omegan1-Omegap1)^2\
                + r^2/(3*e^nu)*Ln^2 + h2
Eq_43_1_check = Eq_43_1 - Eq_43_1_paper
for ii in range(len(sus_func)):
    Eq_43_1 = Eq_43_1.subs(sus_func[ii])
    Eq_43_1_paper = Eq_43_1_paper.subs(sus_func[ii])
    Eq_43_1_check = Eq_43_1_check.subs(sus_func[ii])

Eq_43_1 = Eq_43_1.simplify_full()

for ii in range(len(sus_legendre)):
    Eq_43_1 = Eq_43_1.subs(sus_legendre[ii])
    Eq_43_1_paper = Eq_43_1_paper.subs(sus_legendre[ii])

Eq_43_1 = Eq_43_1.simplify_full()
Eq_43_1 = (Eq_43_1 - Eq_43_1.subs(LP2==0))/LP2
Eq_43_1_paper = (Eq_43_1_paper - Eq_43_1_paper.subs(LP2==0))/LP2
Eq_43_1 = Eq_43_1.simplify_full()

Eq_43_1_paper = Eq_43_1_paper.simplify_full()
Eq_43_1_check = Eq_43_1_check.simplify_full()

show(LatexExpr("\\text{Ecuación (43.1) }\\rightarrow"), bool(Eq_43_1_check==0))

show(EN(Eq_43_1_check))

Ecuación (43.2)

In [ ]:
var('gamman', latex_name = "\\gamma_n")

Eq_43_2 = formal_diff(formal_diff(mue,eps),eps)/muinf
Eq_43_2_paper = + C00*p*Phi2/chie\
                + A00*n*eta2/chie\
                - r^2/(3*e^(nu))*n/chie*(A+p*formal_diff(A,p) + n*p*formal_diff(A,x^2))*(Omegan1-Omegap1)^2\
                + r^2/(3*e^nu)*Lp^2 + h2
Eq_43_2_check = Eq_43_2 - Eq_43_2_paper
for ii in range(len(sus_func)):
    Eq_43_2 = Eq_43_2.subs(sus_func[ii])
    Eq_43_2_paper = Eq_43_2_paper.subs(sus_func[ii])
    Eq_43_2_check = Eq_43_2_check.subs(sus_func[ii])

Eq_43_2 = Eq_43_2.simplify_full()

for ii in range(len(sus_legendre)):
    Eq_43_2 = Eq_43_2.subs(sus_legendre[ii])

Eq_43_2 = Eq_43_2.simplify_full()
Eq_43_2 = (Eq_43_2 - Eq_43_2.subs(LP2==0))/LP2
Eq_43_2 = Eq_43_2.simplify_full()

Eq_43_2_paper = Eq_43_2_paper.simplify_full()
Eq_43_2_paper = (Eq_43_2_paper - Eq_43_2_paper.subs(LP2==0))/LP2
Eq_43_2_paper = Eq_43_2_paper.simplify_full()

Eq_43_2_check = Eq_43_2_check.simplify_full()

show(LatexExpr("\\text{Ecuación (43.2) }\\rightarrow"), bool(Eq_43_2_check==0))

show(EN(Eq_43_2_check))

Ecuación (45)

In [ ]:
ddEq_2233 = ddG[2,2] - kappa*ddT[2,2] - ddG[3,3] + kappa*ddT[3,3] == 0

h2_value = solve(ddEq_2233, h2)[0].rhs()
h2_paper = r^4/(6*e^(nu+lamb))*formal_diff(Ln,r)^2 + 8*pi*r^4/(3*e^nu)*(Psi0-Lambda0)*Ln^2\
         + 8*pi*r^4/(3*e^(nu))*(chie*p*(Omegan1-Omegap1)*(Ln+Lp) - A*n*p*(Omegan1-Omegap1)^2) - v2

for ii in range(len(sus_func)):
    h2_value = h2_value.subs(sus_func[ii])
    h2_paper = h2_paper.subs(sus_func[ii])

h2_value = h2_value.simplify_full()
h2_paper = h2_paper.simplify_full()
for ii in range(len(sus_legendre)):
    h2_value = h2_value.subs(sus_legendre[ii])
    h2_paper = h2_paper.subs(sus_legendre[ii])

h2_value = (h2_value - h2_value.subs(LP2==0))/LP2
h2_paper = (h2_paper - h2_paper.subs(LP2==0))/LP2
h2_check = (h2_value - h2_paper).simplify_full()

show(LatexExpr("h_2 \\rightarrow"), bool(h2_check==0))

show(EN(h2_check))

Ecuación (46)

In [ ]:
ddEq_12 = (ddG[1,2] - kappa*ddT[1,2])*(-r/(3*cos(th)*sin(th)))
ddEq_12_paper = (1/r*(v2+h2) - formal_diff(k2+h2,r) - formal_diff(nu,r)/2*(h2-v2))*2*r
for ii in range(len(sus_func)):
    ddEq_12 = ddEq_12.subs(sus_func[ii])
    ddEq_12_paper = ddEq_12_paper.subs(sus_func[ii])

ddEq_12_check = (ddEq_12 - ddEq_12_paper).simplify_full()

show(LatexExpr("\\text{Ecuación (46) }\\rightarrow"), bool(ddEq_12_check==0))

show(EN(ddEq_12_check))

Ecuación (48)

In [ ]:
ddEq_00 = ddG[0,0] - kappa*ddT[0,0]

ddEq_00_paper = - 16*pi*r^2/(3*e^(nu))*((Psi0 - Lambda0)*Ln^2\
                                      + chie*p*(Omegan1 - Omegap1)*(Ln + Lp)\
                                      - A*n*p*(Omegan1 - Omegap1)^2)\
                + 8*pi*(- Lambda2)\
                - 2/r^2*formal_diff(r/e^(lamb)*v2,r)\
                - r^2/(6*e^(lamb + nu))*(formal_diff(Ln,r))^2\
                + 2/e^(lamb)*(formal_diff(formal_diff(k2,r),r)\
                              +(3/r - formal_diff(lamb,r)/2)*formal_diff(k2,r)\
                              - 2*e^(lamb)/r^2*k2) - 6/r^2*v2

for ii in range(len(sus_func)):
    ddEq_00 = ddEq_00.subs(sus_func[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_func[ii])
    
for ii in range(0,len(sus_descomp),2):
    ddEq_00 = ddEq_00.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_00_paper = ddEq_00_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_00 = ddEq_00.simplify_full()
ddEq_00_paper = ddEq_00_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_00 = ddEq_00.subs(sus_legendre[ii])
    ddEq_00_paper = ddEq_00_paper.subs(sus_legendre[ii])

ddEq_00 = (ddEq_00 - ddEq_00.subs(LP2==0))/LP2
ddEq_00 = ddEq_00.simplify_full()
ddEq_00_check = (ddEq_00 - ddEq_00_paper).simplify_full()

show(LatexExpr("\\text{Ecuación (48) }\\rightarrow"), bool(ddEq_00_check==0))

show(EN(ddEq_00_check))

Ecuación (50)

In [ ]:
ddEq_11 = ddG[1,1] - kappa*ddT[1,1]

ddEq_11_paper = 2/(r*e^(lamb))*formal_diff(h2,r)\
              - 6/r^2*h2 - 2/(r*e^(lamb))*(formal_diff(nu,r)+1/r)*v2\
              + 1/e^(lamb)*(formal_diff(nu,r)+2/r)*formal_diff(k2,r)\
              - 4/r^2*k2 - r^2/(6*e^(lamb + nu))*formal_diff(Ln,r)^2\
              + 8*pi*((Psi0 - Lambda0)*h2 + r^2/(3*e^(nu))*(mue*n*Ln^2 + chie*p*Lp^2)\
                      - r^2/(3*e^(nu))*n*p*A*(Omegan1 - Omegap1)^2)

for ii in range(len(sus_func)):
    ddEq_11 = ddEq_11.subs(sus_func[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_func[ii])

for ii in range(0,len(sus_descomp),2):
    ddEq_11 = ddEq_11.substitute_function(sus_descomp[ii], sus_descomp[ii+1])
    ddEq_11_paper = ddEq_11_paper.substitute_function(sus_descomp[ii], sus_descomp[ii+1])

ddEq_11 = ddEq_11.simplify_full()
ddEq_11_paper = ddEq_11_paper.simplify_full()
for ii in range(len(sus_legendre)):
    ddEq_11 = ddEq_11.subs(sus_legendre[ii])
    ddEq_11_paper = ddEq_11_paper.subs(sus_legendre[ii])

ddEq_11 = (ddEq_11 - ddEq_11.subs(LP2==0))/LP2
ddEq_11 = ddEq_11.simplify_full()
ddEq_11_paper = (ddEq_11_paper - ddEq_11_paper.subs(LP2==0))/LP2
ddEq_11_check = (ddEq_11 - ddEq_11_paper).simplify_full()

show(LatexExpr("\\text{Ecuación (50) }\\rightarrow"), bool(ddEq_11_check==0))

show(EN(ddEq_11_check))